# Day 3

## Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
rows = open("../inputs/3.txt").readlines()
rows = [row.strip() for row in rows]
rows[:3]

In [ ]:
import string

part_symbols = {char for char in string.punctuation if char != "."}
print(part_symbols)

In [ ]:
from IPython.display import Markdown, HTML
from textwrap import dedent


def get_part_num_sum(rows) -> list[int]:
    all_nums = []
    part_nums = []
    for i, row in enumerate(rows):
        display(Markdown(f"# Row {i}"))
        display(Markdown(f"`{row}`"))

        last_position = len(row) - 1

        # flags to determine whether we can check above/below the current row
        top_row = i == 0
        bottom_row = i == last_position

        # build up a string as we find numbers in the row
        checking_number = ""
        for char_index, char in enumerate(row):
            if not char.isdigit():
                checking_number = ""
                continue

            checking_number += char
            # if we're at the end of the row, or the next char isn't a digit, it's time to check
            # our checking_number against values above/below/left/right and diagonal for symbols
            if char_index == last_position or not row[char_index + 1].isdigit():
                print(f"checking {checking_number}...")
                above = i - 1
                below = i + 1
                left = max(0, char_index - len(checking_number))
                right = min(char_index + 1, len(row) - 1)
                print(f"{i=} --> {above=}, {below=}")
                print(f"{char_index=} --> {left=}, {right=}")
                print("-" * 10)

                # check all around the current number, e.g.:
                # xxxxx
                # y123y
                # zzzzz

                chars_above = []
                if not top_row:
                    chars_above = rows[above][left : right + 1]

                chars_below = []
                if not bottom_row:
                    chars_below = rows[below][left : right + 1]

                char_left = row[left]
                if char_left in part_symbols:
                    print("char to the left is a part symbol")
                    part_nums.append(int(checking_number))

                char_right = row[right]
                if char_right in part_symbols:
                    print("char to the right is a part symbol")
                    part_nums.append(int(checking_number))

                if not top_row:
                    print(f"{chars_above=}")
                print(f"{char_left=}, {char_right=}")
                if not bottom_row:
                    print(f"{chars_below=}")

                if any([char in part_symbols for char in chars_above]):
                    part_nums.append(int(checking_number))
                    display(HTML("<mark>chars above have part symbols</mark>"))
                    display(
                        HTML(
                            dedent(
                                f"""<pre>
                        {chars_above}
                        .{checking_number}.
                        </pre>""".strip()
                            )
                        )
                    )

                if any([char in part_symbols for char in chars_below]):
                    part_nums.append(int(checking_number))
                    display(HTML("<mark>chars below have part symbols</mark>"))
                    display(
                        HTML(
                            dedent(
                                f"""<pre>
                        .{checking_number}.
                        {chars_below}
                        </pre>""".strip()
                            )
                        )
                    )

                display(
                    HTML(
                        dedent(
                            f"""
                    <div style="background-color: #111">
                    <pre>
                    {rows[above] if not top_row else ""}
                    {row}
                    {rows[below] if not bottom_row else ""}
                    </pre>
                    </div>""".strip()
                        )
                    )
                )
                all_nums.append(int(checking_number))

    print(f"{all_nums=}")
    print(f"{part_nums=}")
    leftover_nums = list(set(all_nums) - set(part_nums))
    print(f"{leftover_nums=}")
    return part_nums


test_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..""".split(
    "\n"
)
print(test_input[:3])
test_nums = get_part_num_sum(test_input)
test_num_sum = sum(test_nums)
assert test_num_sum == 4361

In [4]:
test_num_sum

4361

In [ ]:
nums = get_part_num_sum(rows)
num_sum = sum(nums)
num_sum

In [6]:
num_sum

546563

## Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
from IPython.display import Markdown, HTML
from textwrap import dedent
from collections import defaultdict


GEAR_SYMBOL = "*"


def get_gear_powers(rows) -> int:
    gear_powers = defaultdict(list)
    for i, row in enumerate(rows):
        display(Markdown(f"# Row {i}"))
        display(Markdown(f"`{row}`"))

        last_position = len(row) - 1

        # flags to determine whether we can check above/below the current row
        top_row = i == 0
        bottom_row = i == last_position

        # build up a string as we find numbers in the row
        checking_number = ""
        for char_index, char in enumerate(row):
            if not char.isdigit():
                checking_number = ""
                continue

            checking_number += char
            # if we're at the end of the row, or the next char isn't a digit, it's time to check
            # our checking_number against values above/below/left/right and diagonal for gears
            if char_index == last_position or not row[char_index + 1].isdigit():
                print(f"checking {checking_number}...")
                above = i - 1
                below = i + 1
                left = max(0, char_index - len(checking_number))
                right = min(char_index + 1, len(row) - 1)
                print(f"{i=} --> {above=}, {below=}")
                print(f"{char_index=} --> {left=}, {right=}")

                # check all around the current number, e.g.:
                # xxxxx
                # y123y
                # zzzzz

                chars_above = []
                if not top_row:
                    chars_above = rows[above][left : right + 1]

                chars_below = []
                if not bottom_row:
                    chars_below = rows[below][left : right + 1]

                char_left = row[left]
                if char_left == GEAR_SYMBOL:
                    print("char to the left is a gear")
                    gear_powers[(i, left)].append(int(checking_number))

                char_right = row[right]
                if char_right == GEAR_SYMBOL:
                    print("char to the right is a gear")
                    gear_powers[(i, right)].append(int(checking_number))

                if GEAR_SYMBOL in chars_above:
                    gear_index = left + chars_above.index(GEAR_SYMBOL)
                    print(f"gear above at {gear_index=}")
                    gear_powers[(above, gear_index)].append(int(checking_number))

                if GEAR_SYMBOL in chars_below:
                    gear_index = left + chars_below.index(GEAR_SYMBOL)
                    print(f"gear below at {gear_index=}")
                    gear_powers[(below, gear_index)].append(int(checking_number))

                display(
                    HTML(
                        dedent(
                            f"""
                    <div style="background-color: #111">
                    <pre>
                    {rows[above] if not top_row else ""}
                    {row}
                    {rows[below] if not bottom_row else ""}
                    </pre>
                    </div>""".strip()
                        )
                    )
                )

        print(f"gear_powers: {dict(gear_powers)}")

    total_power = 0
    for part_numbers in gear_powers.values():
        if len(part_numbers) > 1:
            print(part_numbers)
            power = 1
            for part_number in part_numbers:
                power *= part_number
            total_power += power
    return total_power


test_input2 = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..""".split(
    "\n"
)
print(test_input2[:3])
test_num2 = get_gear_powers(test_input2)
assert test_num2 == 467835

In [8]:
test_num2

467835

In [ ]:
num2 = get_gear_powers(rows)

In [10]:
num2

91031374